# Lets profile and check on the performance

In [ ]:
%pip install -U datasets accelerate transformers sentence-transformers mlflow psutil pynvml
%restart_python

In [ ]:
%sh
# We need to install this extension to the base env
/databricks/python/bin/pip3 install torch_tb_profiler

In [ ]:
import os
import mlflow

db_catalog = 'brian_ml_dev'
db_schema = 'embedding_training'
db_data_volume = 'datasets'
db_volume = 'training_cache'

browser_host = spark.conf.get("spark.databricks.workspaceUrl")
db_host = f"https://{browser_host}"
db_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

os.environ['DATABRICKS_HOST'] = db_host
os.environ['DATABRICKS_TOKEN'] = db_token

experiment_id = 1715231020254790
experiment_path = '/Users/brian.law@databricks.com/finetuning_embeddings'

mlflow.set_experiment(experiment_id=experiment_id)

os.environ['MLFLOW_EXPERIMENT_ID'] = f'{experiment_id}'

For Profiling we will use a separate train script that has profiling correctly configured


In [ ]:
from pyspark.ml.torch.distributor import TorchDistributor

# num processes is the number of gpus that we want to use
distributor = TorchDistributor(num_processes=1, 
                               local_mode=True, use_gpu=True)

args = {"--per_device_batch_size=256",
        "--epochs=1",
        f"--output_dir=/Volumes/{db_catalog}/{db_schema}/{db_volume}"}

with mlflow.start_run() as run:
    
    run_id = run.info.run_id
    os.environ['MLFLOW_RUN_ID'] = run_id
    os.environ['MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING'] = 'true'

    train_obj = distributor.run('scripts/profile_train.py', *args)